In [ ]:
import re
import random


input_file = "inference.txt"
output_file = "cleaned_pairresult.txt"


cleaned_lines = []


with open(input_file, "r", encoding="utf-8") as file:
    lines = file.readlines()

for line in lines:
    
    match = re.search(r'Output:\s*(.*?)\s*\.```', line)
    if match:
        extracted_text = match.group(1).strip()
        cleaned_lines.append(extracted_text)
    else:
        cleaned_lines.append(random.choice(["Yes", "No"]))


with open(output_file, "w", encoding="utf-8") as file:
    file.write("\n".join(cleaned_lines))

print(f"✅ Cleaning completed, results saved to {output_file}")


In [ ]:
import re
import random


input_file = "list-100k-mf.txt"
output_file = "cleaned_pairresult.txt"


cleaned_lines = []


with open(input_file, "r", encoding="utf-8") as file:
    lines = file.readlines()

for line in lines:
   
    match = re.search(r'Output:\s*(.*?)\s*\.```', line)
    if match:
        extracted_text = match.group(1).strip()
        cleaned_lines.append(extracted_text)
    else:
        cleaned_lines.append(random.choice(["Yes", "No"]))  


with open(output_file, "w", encoding="utf-8") as file:
    file.write("\n".join(cleaned_lines))

print(f"✅ Cleaning completed, results saved to {output_file}")


In [ ]:
import re


input_file = "cleaned_pairresult.txt"
output_file = "cleaned_pairresult1.txt"


with open(input_file, "r", encoding="utf-8") as file:
    lines = file.readlines()


replacements = {
    "```Yes": "Yes",
    "```No": "No"
}


processed_lines = []
for line in lines:
    line = line.strip() 
    for pattern, replacement in replacements.items():
        if re.fullmatch(pattern, line): 
            line = replacement
            break  
    processed_lines.append(line)

# 写入新文件
with open(output_file, "w", encoding="utf-8") as file:
    file.write("\n".join(processed_lines) + "\n")  

print(f"✅ Cleaning completed, results saved to {output_file}")

In [ ]:
import json
import csv
import re


pairalltest_file = "pairall.jsonl"
cleaned_results_file = "cleaned_pairresult1.txt"
output_csv_file = "pair_results.csv"


with open(pairalltest_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

total_lines = len(lines)
half_lines = total_lines // 2 


with open(cleaned_results_file, "r", encoding="utf-8") as f:
    cleaned_results = f.readlines()


pattern = r'Question: Would the user prefer "(.*?)" over "(.*?)"\?Hint: Another'


data_rows = []
for idx in range(half_lines):
    json_line = json.loads(lines[idx])  
    text = json_line["inst"]  

    match = re.search(pattern, text)
    if match:
        movie1 = match.group(1)
        movie2 = match.group(2)
    else:
        movie1 = ""
        movie2 = ""

 
    result_forward = cleaned_results[idx].strip() 
    result_reverse = cleaned_results[idx + half_lines].strip() 

    data_rows.append([idx + 1, movie1, movie2, result_forward, result_reverse])


with open(output_csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Line Number", "Movie 1", "Movie 2", "Result Forward", "Result Reverse"])  
    writer.writerows(data_rows) 

print(f"✅ Cleaning completed, results saved to {output_csv_file}")

In [ ]:

import pandas as pd


input_file = "pair_results.csv" 
output_file = "pair_results_with_scores.csv"


df = pd.read_csv(input_file)


INITIAL_SCORES_MOVIE1 = [10, 9, 8, 7, 6] 
INITIAL_SCORES_MOVIE2 = [5, 4, 3, 2, 1]  
BOOST_SCORE = 200  


results = []
for i in range(0, len(df), 5):
    group = df.iloc[i:i+5].copy() 
    if len(group) < 5:
        continue  
    
    
    scores_forward = {}
    scores_reverse = {}

    for idx, row in enumerate(group.iterrows()):
        movie1, movie2 = row[1]["Movie 1"], row[1]["Movie 2"]
        scores_forward[movie1] = INITIAL_SCORES_MOVIE1[idx]
        scores_forward[movie2] = INITIAL_SCORES_MOVIE2[idx]
        scores_reverse[movie1] = INITIAL_SCORES_MOVIE1[idx]
        scores_reverse[movie2] = INITIAL_SCORES_MOVIE2[idx]

    
    for idx, row in group.iterrows():
        movie1, movie2, result_forward, result_reverse = row["Movie 1"], row["Movie 2"], row["Result Forward"], row["Result Reverse"]
        
        
        if result_forward == "Yes":
            scores_forward[movie1] += BOOST_SCORE 
        elif result_forward == "No":
            scores_forward[movie2] += BOOST_SCORE 

        
        if result_forward == "Yes" and result_reverse == "Yes":
            
            scores_reverse[movie1] += BOOST_SCORE / 2
            scores_reverse[movie2] += BOOST_SCORE / 2
        elif result_forward == "No" and result_reverse == "No":
            
            scores_reverse[movie1] += BOOST_SCORE / 2
            scores_reverse[movie2] += BOOST_SCORE / 2
        elif result_forward == "Yes" and result_reverse == "No":
            
            scores_reverse[movie1] += BOOST_SCORE
        elif result_forward == "No" and result_reverse == "Yes":
            
            scores_reverse[movie2] += BOOST_SCORE

    
    for idx, row in group.iterrows():
        movie1, movie2 = row["Movie 1"], row["Movie 2"]
        results.append([
            row["Line Number"],
            movie1, movie2,
            row["Result Forward"], row["Result Reverse"],
            scores_forward[movie1], scores_forward[movie2],
            scores_reverse[movie1], scores_reverse[movie2]
        ])


output_df = pd.DataFrame(results, columns=[
    "Line Number", "Movie 1", "Movie 2", "Result Forward", "Result Reverse",
    "Movie 1 Forward Score", "Movie 2 Forward Score",
    "Movie 1 Reverse Score", "Movie 2 Reverse Score"
])


output_df.to_csv(output_file, index=False)

print(f"✅ Cleaning completed, results saved to {output_file}")


In [ ]:
import pandas as pd


input_file = "pair_results_with_scores.csv"
output_file = "pair_results_with_total_scores.csv"


df = pd.read_csv(input_file)


df["Movie 1 Total Score"] = df["Movie 1 Forward Score"] + df["Movie 1 Reverse Score"]


df["Movie 2 Total Score"] = df["Movie 2 Forward Score"] + df["Movie 2 Reverse Score"]


same_check_results = []
for i in range(0, len(df), 5): 
    group = df.iloc[i:i+5].copy()  
    
    
    total_scores = group["Movie 1 Total Score"].tolist() + group["Movie 2 Total Score"].tolist()

   
    if len(total_scores) != len(set(total_scores)): 
        same_check_results.extend(["same"] * 5)  
    else:
        same_check_results.extend(["no"] * 5)  


df["Same Check"] = same_check_results


df.to_csv(output_file, index=False)

print(f"✅ Cleaning completed, results saved to{output_file}")

In [ ]:
import pandas as pd


input_file = "pair_results_with_scores.csv"
output_file = "pair_results_with_total_scores.csv"


df = pd.read_csv(input_file)


df["Movie 1 Total Score"] = df["Movie 1 Forward Score"] + df["Movie 1 Reverse Score"]


df["Movie 2 Total Score"] = df["Movie 2 Forward Score"] + df["Movie 2 Reverse Score"]


same_check_results = []
for i in range(0, len(df), 5): 
    group = df.iloc[i:i+5].copy()  
    
    
    total_scores = group["Movie 1 Total Score"].tolist() + group["Movie 2 Total Score"].tolist()

  
    if len(total_scores) != len(set(total_scores)):  
        same_check_results.extend(["same"] * 5)  
    else:
        same_check_results.extend(["no"] * 5)  


df["Same Check"] = same_check_results


same_count = same_check_results.count("same")


df.to_csv(output_file, index=False)


print(f"✅ Cleaning completed, results saved to {output_file}")


In [ ]:
import pandas as pd
import random


input_file = "pair_results_with_total_scores.csv"
output_file = "movies_recommendations.csv"


df = pd.read_csv(input_file)


recommendations = []
for i in range(0, len(df), 5):
    group = df.iloc[i:i+5].copy()

    
    movie_scores = {}
    for _, row in group.iterrows():
        movie_scores[row["Movie 1"]] = row["Movie 1 Total Score"]
        movie_scores[row["Movie 2"]] = row["Movie 2 Total Score"]

    
    sorted_movies = sorted(movie_scores.items(), key=lambda x: x[1], reverse=True)

    
    grouped_movies = {}
    for movie, score in sorted_movies:
        grouped_movies.setdefault(score, []).append(movie)

    
    for score in grouped_movies:
        random.shuffle(grouped_movies[score])

    
    final_sorted_movies = []
    for score in sorted(grouped_movies.keys(), reverse=True):
        final_sorted_movies.extend(grouped_movies[score])

    
    top_5_movies = final_sorted_movies[:5]

    
    recommendations.append([i // 5 + 1] + top_5_movies) 


output_df = pd.DataFrame(recommendations, columns=["Line Number", "Movie 1", "Movie 2", "Movie 3", "Movie 4", "Movie 5"])


output_df.to_csv(output_file, index=False)

print(f"✅ Cleaning completed, results saved to {output_file}")

In [8]:
import pandas as pd

def merge_files(user_item_file, movie_info_file, output_file):
    
    user_item_df = pd.read_csv(user_item_file, header=None, names=['user_id', 'item_id'])
    
    
    movie_info_df = pd.read_csv(movie_info_file, sep='|', header=None, names=['item_id', 'movie_name','year','kind'],engine='python', encoding='latin-1')

    
    merged_df = pd.merge(user_item_df, movie_info_df[['item_id', 'movie_name']], on='item_id', how='left')
    
    
    merged_df.insert(0, 'index', range(1, len(merged_df) + 1))
    
    
    final_df = merged_df[['index', 'item_id', 'movie_name']]
    
    
    final_df.to_csv(output_file, index=False, encoding='utf-8')

# Usage
merge_files('MFgt_save_dict1.csv', 'itemnew-processed.csv', 'gt-match-output.csv')

In [9]:

import pandas as pd


movies_recommendations = pd.read_csv('movies_recommendations.csv')
match_output = pd.read_csv('gt-match-output.csv')

# Merge the files based on the 'Line Number' from movies_recommendations and 'index' from match_output
merged_df = pd.merge(movies_recommendations, match_output[['index', 'movie_name']], how='left', left_on='Line Number', right_on='index')

# Drop the extra 'index' column after the merge
merged_df = merged_df.drop(columns=['index'])

# Save the merged result to a new CSV file
output_file = 'merged_with_movie_name.csv'
merged_df.to_csv(output_file, index=False)

print(f"CSV file created at: {output_file}")


CSV file created at: merged_with_movie_name.csv


In [10]:

import pandas as pd

# Load the merged CSV file
merged_df = pd.read_csv('merged_with_movie_name.csv')

# Function to check if the movie_name matches any of Movie1 to Movie5
def check_match(row):
    for i in range(1, 6):
        if row['movie_name'] == row[f'Movie {i}']:
            return i
    return 0

# Apply the check_match function to each row and create a new column for the result
merged_df['Match'] = merged_df.apply(check_match, axis=1)

# Save the updated dataframe with the new column
output_file_with_match = 'merged_with_match_column.csv'
merged_df.to_csv(output_file_with_match, index=False)

print(f"CSV file created at: {output_file_with_match}")


CSV file created at: merged_with_match_column.csv


In [ ]:

import pandas as pd
import numpy as np

# Load the merged CSV file with the match column
df = pd.read_csv('merged_with_match_column.csv')

# Function to calculate Hit@n
def calculate_hit(df, n):
    return df['Match'].apply(lambda x: 1 if 0 < x <= n else 0).mean()

# Function to calculate DCG@n and NDCG@n for each row
def calculate_dcg(row, n):
    match = row['Match']
    if 0 < match <= n:  # if there's a match in the top n
        return 1 / np.log2(match + 1)
    return 0

# Function to calculate NDCG@n
def calculate_ndcg(df, n):
    # Calculate DCG@n for each row
    df[f'DCG@{n}'] = df.apply(lambda row: calculate_dcg(row, n), axis=1)
    # Ideal DCG@n (IDCG@n) is always 1, since the best match is in the first position
    idcg = 1 / np.log2(2)
    # Calculate NDCG@n for each row
    df[f'NDCG@{n}'] = df[f'DCG@{n}'] / idcg
    return df[f'NDCG@{n}'].mean()

# Calculate Hit@3 and NDCG@3
hit_at_3 = calculate_hit(df, 3)
ndcg_at_3 = calculate_ndcg(df, 3)

# Calculate Hit@5 and NDCG@5
hit_at_5 = calculate_hit(df, 5)
ndcg_at_5 = calculate_ndcg(df, 5)

# Scale factor
scale_factor = 70 / 1777

# Scaled results
scaled_hit_at_3 = hit_at_3 * scale_factor
scaled_ndcg_at_3 = ndcg_at_3 * scale_factor
scaled_hit_at_5 = hit_at_5 * scale_factor
scaled_ndcg_at_5 = ndcg_at_5 * scale_factor

# Output the results
print(f"Hit@3: {hit_at_3}")
print(f"NDCG@3: {ndcg_at_3}")
print(f"Hit@5: {hit_at_5}")
print(f"NDCG@5: {ndcg_at_5}")

print("\n### Scaled Results (Multiplied by 70 / 1777) ###")
print(f"Scaled Hit@3: {scaled_hit_at_3}")
print(f"Scaled NDCG@3: {scaled_ndcg_at_3}")
print(f"Scaled Hit@5: {scaled_hit_at_5}")
print(f"Scaled NDCG@5: {scaled_ndcg_at_5}")
